<a href="https://colab.research.google.com/github/Khushil-Modi/SJSU/blob/main/CMPE%20297/Assignment%202/CMPE_297_Assignment_2(c)_Link_Prediction_of_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing Libraries

In [27]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!pip install torch_sparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
!pip install torch_scatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importing Libraries

In [39]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])

## Data Preparation

In [42]:
dataset = 'Cora'
path = osp.join('.', 'data', dataset)
#path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Planetoid')
dataset = Planetoid(path, name='Cora', transform=transform)

In [43]:
# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.
train_data, val_data, test_data = dataset[0]

##Training the Model

In [44]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [45]:
model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [46]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss

Training the model

In [47]:
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

Epoch: 001, Loss: 0.6930, Val: 0.7007, Test: 0.6935
Epoch: 002, Loss: 0.6802, Val: 0.6933, Test: 0.6887
Epoch: 003, Loss: 0.7094, Val: 0.6932, Test: 0.6955
Epoch: 004, Loss: 0.6747, Val: 0.6991, Test: 0.7176
Epoch: 005, Loss: 0.6856, Val: 0.7607, Test: 0.7772
Epoch: 006, Loss: 0.6901, Val: 0.7302, Test: 0.7539
Epoch: 007, Loss: 0.6916, Val: 0.6285, Test: 0.6551
Epoch: 008, Loss: 0.6920, Val: 0.6110, Test: 0.6320
Epoch: 009, Loss: 0.6917, Val: 0.6297, Test: 0.6482
Epoch: 010, Loss: 0.6906, Val: 0.6496, Test: 0.6667
Epoch: 011, Loss: 0.6884, Val: 0.6662, Test: 0.6782
Epoch: 012, Loss: 0.6847, Val: 0.6774, Test: 0.6856
Epoch: 013, Loss: 0.6800, Val: 0.6865, Test: 0.6922
Epoch: 014, Loss: 0.6777, Val: 0.6917, Test: 0.7023
Epoch: 015, Loss: 0.6757, Val: 0.6882, Test: 0.7108
Epoch: 016, Loss: 0.6703, Val: 0.6838, Test: 0.7165
Epoch: 017, Loss: 0.6642, Val: 0.6810, Test: 0.7198
Epoch: 018, Loss: 0.6598, Val: 0.6847, Test: 0.7250
Epoch: 019, Loss: 0.6547, Val: 0.6942, Test: 0.7346
Epoch: 020, 

In [48]:
print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Final Test: 0.9017


#References:

https://github.com/pyg-team/pytorch_geometric/blob/master/examples/link_pred.py

https://colab.research.google.com/drive/1f2KVSVsk5f4dpZlxNE0RBIHowqDs3KSp#scrollTo=dQ98qUOAt5JK

http://cs230.stanford.edu/projects_spring_2020/reports/38854344.pdf